In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import librosa
from data_loading import extract_features
import os
import pickle
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def create_base_network(input_shape):
    """Shared feature extractor."""
    model = tf.keras.Sequential([
        layers.Conv1D(64, kernel_size=5, activation=None, input_shape=input_shape),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling1D(pool_size=2),

        layers.Conv1D(128, kernel_size=5, activation=None, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling1D(pool_size=2),

        layers.Conv1D(256, kernel_size=5, activation=None, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.1),
        layers.GlobalMaxPooling1D(),

        layers.Dense(128, activation=None, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.1),
        layers.Dropout(0.5),

        layers.Dense(64, activation=None, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.1),
        layers.Dropout(0.5)
    ])
    return model

def create_siamese_network(input_shape):
    base_network = create_base_network(input_shape)
    
    # Two inputs for the Siamese network
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)
    
    # Shared embedding layers
    embedding_a = base_network(input_a)
    embedding_b = base_network(input_b)

    # L2 normalization of embeddings
    embedding_a = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(embedding_a)
    embedding_b = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(embedding_b)

    # Distance metric (Euclidean Distance)
    distance = layers.Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1, keepdims=True) + 1e-8))([embedding_a, embedding_b])

    
    return Model(inputs=[input_a, input_b], outputs=distance)

def contrastive_loss(y_true, y_pred, margin=1):
    """Contrastive loss as a custom loss function."""
    # y_true = tf.cast(tf.reshape(y_true, (-1, 1)), tf.float32)  # Reshape to [batch_size, 1]
    y_true = tf.cast(y_true, tf.float32)
    squared_distance = tf.square(y_pred)
    margin_distance = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean((1 - y_true) * squared_distance + y_true * margin_distance)

def create_pairs(features, labels):
    """Generate pairs: (input_1, input_2, label)."""
    pairs = []
    pair_labels = []
    for i in range(len(features)):
        for j in range(i, len(features)):
            # Create a positive pair (same speaker)
            if labels[i] == labels[j]:
                pairs.append((features[i], features[j]))
                pair_labels.append(0)  # Similar
            # Create a negative pair (different speakers)
            else:
                pairs.append((features[i], features[j]))
                pair_labels.append(1)  # Dissimilar
    pairs, pair_labels = np.array(pairs), np.array(pair_labels)
    indices = np.random.permutation(len(pairs))
    pairs, pair_labels = pairs[indices], pair_labels[indices]
    return np.array(pairs), np.array(pair_labels)

# 6. Training
def train_siamese_network():
    siamese_net = create_siamese_network((80000,14))
    print("Model Created")
    optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)

    siamese_net.compile(optimizer=optimizer, loss=contrastive_loss, metrics=['accuracy'])
    
    for file in os.listdir("dataset"):
        if file.startswith("features_"):
            with open(f"dataset/{file}", "rb") as f:
                features = pickle.load(f)
            part_number=file.split(".")[0].split("_")[-1]
            with open(f"dataset/labels_part_{part_number}.pkl", "rb") as f:
                labels=pickle.load(f)
            print("Features Loaded for Part ", part_number)
            # Break Features and Labels into 10 parts
            features = np.array(features)
            labels = np.array(labels)
            
            # Normalize Features
            features = (features - np.mean(features, axis=0)) / (np.std(features, axis=0) + 1e-8)

            features = np.array_split(features, 20)
            labels = np.array_split(labels, 20)
            for features_part, labels_part in zip(features, labels):
                pairs, pair_labels = create_pairs(features_part, labels_part)
                pairs = np.array(pairs)
                split = int(len(pairs) * 0.8)
                train_pairs, test_pairs = pairs[:split], pairs[split:]
                train_labels, test_labels = pair_labels[:split], pair_labels[split:]
                history=siamese_net.fit(
                    [train_pairs[:, 0], train_pairs[:, 1]], train_labels,
                    validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels),
                    epochs=1,
                    batch_size=16,
                    verbose=0
                )
                print("Model Trained")
            siamese_net.save(f"weights/siamese_model_part_{part_number}.h5")
            print("Model Saved")
                
                # Infer on 10 test pairs
                # n1=random.randint(0, len(test_pairs)-11)
                # n2=n1+10
                # # Shuffle the test pairs and labels
                # test_pairs = np.array(test_pairs)
                # test_labels = np.array(test_labels)
                # perm=np.random.permutation(len(test_pairs))
                # test_pairs = test_pairs[perm]
                # test_labels = test_labels[perm]
                
                # test_pairs = test_pairs[n1:n2]
                # test_labels = test_labels[n1:n2]
                # test_distances = siamese_net.predict([test_pairs[:, 0], test_pairs[:, 1]])
                # test_loss=contrastive_loss(test_labels, test_distances)
                # # roc_auc = roc_auc_score(test_labels, test_distances)
                # threshold = 0.5
                # test_predictions=[]
                # for t in test_distances:
                #     test_predictions.append(0 if t.mean()<threshold else 1)
                # test_predictions = (test_distances > threshold).astype(int)
                # test_predictions = test_distances.ravel() < threshold
                
                # precision=precision_score(test_labels, test_predictions)
                # recall=recall_score(test_labels, test_predictions)
                # f1=f1_score(test_labels, test_predictions)
                
                # test_predictions = np.squeeze(test_predictions)  # Remove extra dimensions
                
                # # print(test_distances, test_labels, test_predictions)
                # # print(test_labels, test_predictions)
                # # print(test_distances.shape, test_labels.shape, test_predictions.shape)
                # print(f"Test Loss: {test_loss}")
                # print("Test Accuracy: ", np.mean(test_predictions == test_labels))
                # print(f"ROC AUC: {roc_auc}")
                # print(f"Precision: {precision}")
                # print(f"Recall: {recall}")
                # print(f"F1 Score: {f1}")
                # print("Model Evaluated")                
                
    return siamese_net

In [3]:
# 7. Run Training
model = train_siamese_network()

C:\Users\Innomer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Innomer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Model Created
Features Loaded for Part  0
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


Model Trained
Model Saved
Features Loaded for Part  1
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


Model Trained
Model Saved
Features Loaded for Part  10
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


Model Trained
Model Saved
Features Loaded for Part  100
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


...

Model Trained
Model Saved
Features Loaded for Part  27
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


Model Trained
Model Saved
Features Loaded for Part  28
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


Model Trained
Model Saved
Features Loaded for Part  29
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained
Model Trained


Model Trained
Model Saved
Features Loaded for Part  3


KeyboardInterrupt: 